#### Loading dataset using pandas

In [26]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [27]:
data = pd.read_csv(filepath_or_buffer='../data/melbourne-housing-snapshot/melb_data.csv')
y = data.Price
melb_predictors = data.drop(['Price'], axis=1) # Drop column that has to be predicted
X = melb_predictors.select_dtypes(exclude=['object'])

In [28]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2, random_state=1)

#### Definition of score computation function

In [29]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

def score_dataset(X_train, X_valid, y_train, y_valid):
    model = RandomForestRegressor(n_estimators=10, random_state=1)
    model.fit(X=X_train, y=y_train)
    preds = model.predict(X_valid)
    return mean_absolute_error(y_true=y_valid, y_pred=preds)

#### Missing values: Approach 1 (Drop columns)

In [30]:
cols_with_missing = [col for col in X_train.columns if X_train[col].isnull().any()]
reduced_X_train = X_train.drop(cols_with_missing, axis=1)
reduced_X_valid = X_valid.drop(cols_with_missing, axis=1)
print(f"MAE from Approach 1 (Drop columns): {score_dataset(reduced_X_train, reduced_X_valid, y_train, y_valid)}")

MAE from Approach 1 (Drop columns): 187082.57548478153


#### Missing values: Approach 2 (Imputation)

In [31]:
from sklearn.impute import SimpleImputer

my_imputer = SimpleImputer()
imputed_X_train = pd.DataFrame(my_imputer.fit_transform(X_train))
imputed_X_valid = pd.DataFrame(my_imputer.transform(X_valid))
imputed_X_train.columns = X_train.columns
imputed_X_valid.columns = X_valid.columns
print(f"MAE from Approach 2 (Imputation): {score_dataset(imputed_X_train, imputed_X_valid, y_train, y_valid)}")

MAE from Approach 2 (Imputation): 174917.69150711832


#### Missing values: Approach 3 (Imputation extension)

In [32]:
X_train_plus = X_train.copy()
X_valid_plus = X_valid.copy()

for col in cols_with_missing:
    X_train_plus[col + 'was_missing'] = X_train_plus[col].isnull()
    X_valid_plus[col + 'was_missing'] = X_valid_plus[col].isnull()

my_imputer = SimpleImputer()
imputed_X_train_plus = pd.DataFrame(my_imputer.fit_transform(X_train_plus))
imputed_X_valid_plus = pd.DataFrame(my_imputer.transform(X_valid_plus))
imputed_X_train_plus.columns = X_train_plus.columns
imputed_X_valid_plus.columns = X_valid_plus.columns
print(f"MAE from Approach 3 (Imputation extension): {score_dataset(imputed_X_train_plus, imputed_X_valid_plus, y_train, y_valid)}")

MAE from Approach 3 (Imputation extension): 177346.30320324007
